In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import torch

In [2]:
wadi_sensors = [
    "Row", "Date", "Time", "1_AIT_001_PV", "1_AIT_002_PV", "1_AIT_003_PV",
    "1_AIT_004_PV", "1_AIT_005_PV", "1_FIT_001_PV", "1_LS_001_AL", "1_LS_002_AL",
    "1_LT_001_PV", "1_MV_001_STATUS", "1_MV_002_STATUS", "1_MV_003_STATUS",
    "1_MV_004_STATUS", "1_P_001_STATUS", "1_P_002_STATUS", "1_P_003_STATUS",
    "1_P_004_STATUS", "1_P_005_STATUS", "1_P_006_STATUS", "2_DPIT_001_PV",
    "2_FIC_101_CO", "2_FIC_101_PV", "2_FIC_101_SP", "2_FIC_201_CO", "2_FIC_201_PV",
    "2_FIC_201_SP", "2_FIC_301_CO", "2_FIC_301_PV", "2_FIC_301_SP", "2_FIC_401_CO",
    "2_FIC_401_PV", "2_FIC_401_SP", "2_FIC_501_CO", "2_FIC_501_PV", "2_FIC_501_SP",
    "2_FIC_601_CO", "2_FIC_601_PV", "2_FIC_601_SP", "2_FIT_001_PV", "2_FIT_002_PV",
    "2_FIT_003_PV", "2_FQ_101_PV", "2_FQ_201_PV", "2_FQ_301_PV", "2_FQ_401_PV",
    "2_FQ_501_PV", "2_FQ_601_PV", "2_LS_001_AL", "2_LS_002_AL", "2_LS_101_AH",
    "2_LS_101_AL", "2_LS_201_AH", "2_LS_201_AL", "2_LS_301_AH", "2_LS_301_AL",
    "2_LS_401_AH", "2_LS_401_AL", "2_LS_501_AH", "2_LS_501_AL", "2_LS_601_AH",
    "2_LS_601_AL", "2_LT_001_PV", "2_LT_002_PV", "2_MCV_007_CO", "2_MCV_101_CO",
    "2_MCV_201_CO", "2_MCV_301_CO", "2_MCV_401_CO", "2_MCV_501_CO", "2_MCV_601_CO",
    "2_MV_001_STATUS", "2_MV_002_STATUS", "2_MV_003_STATUS", "2_MV_004_STATUS",
    "2_MV_005_STATUS", "2_MV_006_STATUS", "2_MV_009_STATUS", "2_MV_101_STATUS",
    "2_MV_201_STATUS", "2_MV_301_STATUS", "2_MV_401_STATUS", "2_MV_501_STATUS",
    "2_MV_601_STATUS", "2_P_001_STATUS", "2_P_002_STATUS", "2_P_003_SPEED",
    "2_P_003_STATUS", "2_P_004_SPEED", "2_P_004_STATUS", "2_PIC_003_CO",
    "2_PIC_003_PV", "2_PIC_003_SP", "2_PIT_001_PV", "2_PIT_002_PV", "2_PIT_003_PV",
    "2_SV_101_STATUS", "2_SV_201_STATUS", "2_SV_301_STATUS", "2_SV_401_STATUS",
    "2_SV_501_STATUS", "2_SV_601_STATUS", "2A_AIT_001_PV", "2A_AIT_002_PV",
    "2A_AIT_003_PV", "2A_AIT_004_PV", "2B_AIT_001_PV", "2B_AIT_002_PV",
    "2B_AIT_003_PV", "2B_AIT_004_PV", "3_AIT_001_PV", "3_AIT_002_PV",
    "3_AIT_003_PV", "3_AIT_004_PV", "3_AIT_005_PV", "3_FIT_001_PV", "3_LS_001_AL",
    "3_LT_001_PV", "3_MV_001_STATUS", "3_MV_002_STATUS", "3_MV_003_STATUS",
    "3_P_001_STATUS", "3_P_002_STATUS", "3_P_003_STATUS", "3_P_004_STATUS",
    "LEAK_DIFF_PRESSURE", "PLANT_START_STOP_LOG", "TOTAL_CONS_REQUIRED_FLOW"
]
swat_sensors = [
    "Timestamp", "FIT101", "LIT101", "MV101", "P101", "P102", "AIT201", "AIT202",
    "AIT203", "FIT201", "MV201", "P201", "P202", "P203", "P204", "P205", "P206",
    "DPIT301", "FIT301", "LIT301", "MV301", "MV302", "MV303", "MV304", "P301",
    "P302", "AIT401", "AIT402", "FIT401", "LIT401", "P401", "P402", "P403", "P404",
    "UV401", "AIT501", "AIT502", "AIT503", "AIT504", "FIT501", "FIT502", "FIT503",
    "FIT504", "P501", "P502", "PIT501", "PIT502", "PIT503", "FIT601", "P601",
    "P602", "P603", "Normal/Attack"
]

# Create a dictionary with index as key and [sensor_name, type, group] as value
swat_sensors_dict = {
    0: ["FIT101", "Sensor", 1],
    1: ["LIT101", "Sensor", 1],
    2: ["MV101", "Actuator", 1],
    3: ["P101", "Actuator", 1],
    4: ["P102", "Actuator", 1],
    5: ["AIT201", "Sensor", 2],
    6: ["AIT202", "Sensor", 2],
    7: ["AIT203", "Sensor", 2],
    8: ["FIT201", "Sensor", 2],
    9: ["MV201", "Actuator", 2],
    10: ["P201", "Actuator", 2],
    11: ["P202", "Actuator", 2],
    12: ["P203", "Actuator", 2],
    13: ["P204", "Actuator", 2],
    14: ["P205", "Actuator", 2],
    15: ["P206", "Actuator", 2],
    16: ["DPIT301", "Sensor", 3],
    17: ["FIT301", "Sensor", 3],
    18: ["LIT301", "Sensor", 3],
    19: ["MV301", "Actuator", 3],
    20: ["MV302", "Actuator", 3],
    21: ["MV303", "Actuator", 3],
    22: ["MV304", "Actuator", 3],
    23: ["P301", "Actuator", 3],
    24: ["P302", "Actuator", 3],
    25: ["AIT401", "Sensor", 4],
    26: ["AIT402", "Sensor", 4],
    27: ["FIT401", "Sensor", 4],
    28: ["LIT401", "Actuator", 4],  # Note: LIT401 is listed as an Actuator in your data, though LIT typically indicates a sensor
    29: ["P401", "Actuator", 4],
    30: ["P402", "Actuator", 4],
    31: ["P403", "Actuator", 4],
    32: ["P404", "Actuator", 4],
    33: ["UV401", "Actuator", 4],
    34: ["AIT501", "Sensor", 5],
    35: ["AIT502", "Sensor", 5],
    36: ["AIT503", "Sensor", 5],
    37: ["AIT504", "Sensor", 5],
    38: ["FIT501", "Sensor", 5],
    39: ["FIT502", "Sensor", 5],
    40: ["FIT503", "Sensor", 5],
    41: ["FIT504", "Sensor", 5],
    42: ["P501", "Actuator", 5],
    43: ["P502", "Actuator", 5],
    44: ["PIT501", "Sensor", 5],
    45: ["PIT502", "Sensor", 5],
    46: ["PIT503", "Sensor", 5],
    47: ["FIT601", "Sensor", 6],
    48: ["P601", "Actuator", 6],
    49: ["P602", "Actuator", 6],
    50: ["P603", "Actuator", 6]
}

wadi_dict = {
    # Primary Grid (P1)
    0: ["1_AIT_001_PV", "Sensor", "P1"],  # Water quality sensor
    1: ["1_AIT_002_PV", "Sensor", "P1"],  # Water quality sensor
    2: ["1_AIT_003_PV", "Sensor", "P1"],  # Water quality sensor
    3: ["1_AIT_004_PV", "Sensor", "P1"],  # Water quality sensor
    4: ["1_AIT_005_PV", "Sensor", "P1"],  # Water quality sensor
    5: ["1_FIT_001_PV", "Sensor", "P1"],  # Flow meter
    6: ["1_LS_001_AL", "Sensor", "P1"],   # Level switch alarm
    7: ["1_LS_002_AL", "Sensor", "P1"],   # Level switch alarm
    8: ["1_LT_001_PV", "Sensor", "P1"],   # Level transmitter
    9: ["1_MV_001_STATUS", "Actuator", "P1"],  # Motorized valve
    10: ["1_MV_002_STATUS", "Actuator", "P1"],  # Motorized valve
    11: ["1_MV_003_STATUS", "Actuator", "P1"],  # Motorized valve
    12: ["1_MV_004_STATUS", "Actuator", "P1"],  # Motorized valve
    13: ["1_P_001_STATUS", "Actuator", "P1"],   # Pump
    14: ["1_P_002_STATUS", "Actuator", "P1"],   # Pump
    15: ["1_P_003_STATUS", "Actuator", "P1"],   # Pump
    16: ["1_P_004_STATUS", "Actuator", "P1"],   # Pump
    17: ["1_P_005_STATUS", "Actuator", "P1"],   # Pump
    18: ["1_P_006_STATUS", "Actuator", "P1"],   # Pump

    # Secondary Grid Part A (P2A - Elevated Reservoir)
    19: ["2_DPIT_001_PV", "Sensor", "P2A"],  # Differential pressure indicator
    20: ["2_FIT_001_PV", "Sensor", "P2A"],   # Flow meter
    21: ["2_FIT_002_PV", "Sensor", "P2A"],   # Flow meter
    22: ["2_FIT_003_PV", "Sensor", "P2A"],   # Flow meter
    23: ["2_LT_001_PV", "Sensor", "P2A"],    # Level transmitter - elevated reservoir
    24: ["2_LT_002_PV", "Sensor", "P2A"],    # Level transmitter - elevated reservoir
    25: ["2_MV_001_STATUS", "Actuator", "P2A"],  # Motorized valve
    26: ["2_MV_002_STATUS", "Actuator", "P2A"],  # Motorized valve
    27: ["2_MV_003_STATUS", "Actuator", "P2A"],  # Motorized valve
    28: ["2_MV_004_STATUS", "Actuator", "P2A"],  # Motorized valve
    29: ["2_MV_005_STATUS", "Actuator", "P2A"],  # Motorized valve
    30: ["2_MV_006_STATUS", "Actuator", "P2A"],  # Motorized valve
    31: ["2_MV_009_STATUS", "Actuator", "P2A"],  # Motorized valve
    32: ["2_P_001_STATUS", "Actuator", "P2A"],   # Pump
    33: ["2_P_002_STATUS", "Actuator", "P2A"],   # Pump
    34: ["2_PIT_001_PV", "Sensor", "P2A"],       # Pressure indicator
    35: ["2_PIT_002_PV", "Sensor", "P2A"],       # Pressure indicator
    36: ["2_PIT_003_PV", "Sensor", "P2A"],       # Pressure indicator
    37: ["2A_AIT_001_PV", "Sensor", "P2A"],      # Water quality sensor
    38: ["2A_AIT_002_PV", "Sensor", "P2A"],      # Water quality sensor
    39: ["2A_AIT_003_PV", "Sensor", "P2A"],      # Water quality sensor
    40: ["2A_AIT_004_PV", "Sensor", "P2A"],      # Water quality sensor

    # Secondary Grid Part B (P2B - Consumer Tanks)
    41: ["2_FIC_101_PV", "Sensor", "P2B"],       # Flow indicator/controller
    42: ["2_FIC_201_PV", "Sensor", "P2B"],       # Flow indicator/controller
    43: ["2_FIC_301_PV", "Sensor", "P2B"],       # Flow indicator/controller
    44: ["2_FIC_401_PV", "Sensor", "P2B"],       # Flow indicator/controller
    45: ["2_FIC_501_PV", "Sensor", "P2B"],       # Flow indicator/controller
    46: ["2_FIC_601_PV", "Sensor", "P2B"],       # Flow indicator/controller
    47: ["2_FQ_101_PV", "Sensor", "P2B"],        # Flow totalizer
    48: ["2_FQ_201_PV", "Sensor", "P2B"],        # Flow totalizer
    49: ["2_FQ_301_PV", "Sensor", "P2B"],        # Flow totalizer
    50: ["2_FQ_401_PV", "Sensor", "P2B"],        # Flow totalizer
    51: ["2_FQ_501_PV", "Sensor", "P2B"],        # Flow totalizer
    52: ["2_FQ_601_PV", "Sensor", "P2B"],        # Flow totalizer
    53: ["2_LS_101_AH", "Sensor", "P2B"],        # Level switch high alarm
    54: ["2_LS_101_AL", "Sensor", "P2B"],        # Level switch low alarm
    55: ["2_LS_201_AH", "Sensor", "P2B"],        # Level switch high alarm
    56: ["2_LS_201_AL", "Sensor", "P2B"],        # Level switch low alarm
    57: ["2_LS_301_AH", "Sensor", "P2B"],        # Level switch high alarm
    58: ["2_LS_301_AL", "Sensor", "P2B"],        # Level switch low alarm
    59: ["2_LS_401_AH", "Sensor", "P2B"],        # Level switch high alarm
    60: ["2_LS_401_AL", "Sensor", "P2B"],        # Level switch low alarm
    61: ["2_LS_501_AH", "Sensor", "P2B"],        # Level switch high alarm
    62: ["2_LS_501_AL", "Sensor", "P2B"],        # Level switch low alarm
    63: ["2_LS_601_AH", "Sensor", "P2B"],        # Level switch high alarm
    64: ["2_LS_601_AL", "Sensor", "P2B"],        # Level switch low alarm
    65: ["2_MCV_101_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    66: ["2_MCV_201_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    67: ["2_MCV_301_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    68: ["2_MCV_401_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    69: ["2_MCV_501_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    70: ["2_MCV_601_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    71: ["2_SV_101_STATUS", "Actuator", "P2B"],  # Solenoid valve
    72: ["2_SV_201_STATUS", "Actuator", "P2B"],  # Solenoid valve
    73: ["2_SV_301_STATUS", "Actuator", "P2B"],  # Solenoid valve
    74: ["2_SV_401_STATUS", "Actuator", "P2B"],  # Solenoid valve
    75: ["2_SV_501_STATUS", "Actuator", "P2B"],  # Solenoid valve
    76: ["2_SV_601_STATUS", "Actuator", "P2B"],  # Solenoid valve
    77: ["2B_AIT_001_PV", "Sensor", "P2B"],      # Water quality sensor
    78: ["2B_AIT_002_PV", "Sensor", "P2B"],      # Water quality sensor
    79: ["2B_AIT_003_PV", "Sensor", "P2B"],      # Water quality sensor
    80: ["2B_AIT_004_PV", "Sensor", "P2B"],      # Water quality sensor

    # Return Water Grid (P3)
    81: ["3_AIT_001_PV", "Sensor", "P3"],        # Water quality sensor
    82: ["3_AIT_002_PV", "Sensor", "P3"],        # Water quality sensor
    83: ["3_AIT_003_PV", "Sensor", "P3"],        # Water quality sensor
    84: ["3_AIT_004_PV", "Sensor", "P3"],        # Water quality sensor
    85: ["3_AIT_005_PV", "Sensor", "P3"],        # Water quality sensor
    86: ["3_FIT_001_PV", "Sensor", "P3"],        # Flow meter
    87: ["3_LS_001_AL", "Sensor", "P3"],         # Level switch alarm
    88: ["3_LT_001_PV", "Sensor", "P3"],         # Level transmitter
    89: ["3_MV_001_STATUS", "Actuator", "P3"],   # Motorized valve
    90: ["3_MV_002_STATUS", "Actuator", "P3"],   # Motorized valve
    91: ["3_MV_003_STATUS", "Actuator", "P3"],   # Motorized valve
    92: ["3_P_001_STATUS", "Actuator", "P3"],    # Pump
    93: ["3_P_002_STATUS", "Actuator", "P3"],    # Pump
    94: ["3_P_003_STATUS", "Actuator", "P3"],    # Pump
    95: ["3_P_004_STATUS", "Actuator", "P3"],    # Pump
}
# Example usage: access information about a sensor
# print(swat_sensors_dict[38])  # Output: ['FIT501', 'Sensor', 5]

In [3]:
def load_correlation_matrices_to_tensor():
    file_paths = sorted(glob.glob("/home/berentzenaej/baselines/CST-GL/ADP_*.txt"))
    first_matrix = np.loadtxt(file_paths[0])
    tensor = np.zeros((len(file_paths), *first_matrix.shape))

    for i, path in enumerate(file_paths):
        tensor[i] = np.loadtxt(path)

    return tensor

In [12]:
correlation_tensor=load_correlation_matrices_to_tensor()
print(correlation_tensor.shape)
print(torch.count_nonzero(torch.tensor(correlation_tensor)))

(3, 51, 51)
tensor(1929)


In [8]:
import numpy as np
from scipy.spatial.distance import cosine

def column_consistency(tensor):
    n_matrices = tensor.shape[0]
    n_nodes = tensor.shape[1]

    # Store average similarity for each column
    column_similarities = []
    valid_columns = 0

    # Process each column
    for col_idx in range(n_nodes):
        # Extract the same column from all matrices
        column_vectors = [tensor[matrix_idx, :, col_idx] for matrix_idx in range(n_matrices)]

        # Skip columns that are all zeros or constant values
        if D or \
           any(np.all(vec == vec[0]) for vec in column_vectors):
            continue

        valid_columns += 1

        # Calculate pairwise cosine similarities for this column
        col_cosine_sims = []
        for i in range(n_matrices):
            for j in range(i+1, n_matrices):
                try:
                    cos_sim = 1 - cosine(column_vectors[i], column_vectors[j])
                    col_cosine_sims.append(cos_sim)
                except:
                    # Skip pairs that cause computational issues
                    continue

        # Average similarity for this column (if we have valid comparisons)
        if col_cosine_sims:
            avg_col_sim = np.mean(col_cosine_sims)
            column_similarities.append(avg_col_sim)

    # Average of all column similarities
    overall_avg_similarity = np.mean(column_similarities) if column_similarities else 0.0

    print(f"Valid columns for comparison: {valid_columns} out of {n_nodes}")

    return column_similarities, overall_avg_similarity

# Usage
column_sims, overall_sim = column_consistency(correlation_tensor)
print(f"Overall structural consistency: {overall_sim:.4f}")
print(f"Column-wise consistencies (sample): {[round(sim, 4) for sim in column_sims[:5] if column_sims]}")

Valid columns for comparison: 2 out of 51
Overall structural consistency: 0.7533
Column-wise consistencies (sample): [np.float64(0.84), np.float64(0.6667)]


In [7]:
nan_mask = np.isnan(correlation_tensor)
print(sum(sum(sum(nan_mask))))

0


In [13]:
import numpy as np
from scipy.spatial.distance import cosine

def column_consistency(tensor):
    n_matrices = tensor.shape[0]
    n_nodes = tensor.shape[1]

    # Store average similarity for each column
    column_similarities = []
    valid_columns = 0

    # Diagnostic counters
    zero_vectors = 0
    constant_vectors = 0

    # Process each column
    for col_idx in range(n_nodes):
        # Extract the same column from all matrices
        column_vectors = [tensor[matrix_idx, :, col_idx] for matrix_idx in range(n_matrices)]

        # Count and print detailed diagnostics for this column
        has_zero_vector = any(np.all(vec == 0) for vec in column_vectors)
        has_constant_vector = any(np.all(vec == vec[0]) and not np.all(vec == 0) for vec in column_vectors)

        if has_zero_vector:
            zero_vectors += 1
        if has_constant_vector:
            constant_vectors += 1

        if has_zero_vector or has_constant_vector:
            # Print first few values of each vector for diagnosis
            print(f"Column {col_idx} skipped - zero vectors: {has_zero_vector}, constant vectors: {has_constant_vector}")
            for i, vec in enumerate(column_vectors):
                # Print a sample of the vector and counts of unique values
                unique_vals = np.unique(vec)
                print(f"  Matrix {i} - Unique values: {len(unique_vals)}, Sample: {vec[:5]}")
            continue

        valid_columns += 1

        # Calculate pairwise cosine similarities for this column
        col_cosine_sims = []
        for i in range(n_matrices):
            for j in range(i+1, n_matrices):
                try:
                    cos_sim = 1 - cosine(column_vectors[i], column_vectors[j])
                    col_cosine_sims.append(cos_sim)
                except Exception as e:
                    print(f"Error in column {col_idx}, matrices {i} and {j}: {e}")
                    continue

        # Average similarity for this column
        avg_col_sim = np.mean(col_cosine_sims) if col_cosine_sims else 0.0
        column_similarities.append(avg_col_sim)

    # Overall diagnostics
    print(f"\nDiagnostic Summary:")
    print(f"Columns with zero vectors: {zero_vectors}")
    print(f"Columns with non-zero constant vectors: {constant_vectors}")
    print(f"Valid columns for comparison: {valid_columns} out of {n_nodes}")

    # Average of all column similarities
    overall_avg_similarity = np.mean(column_similarities) if column_similarities else 0.0

    return column_similarities, overall_avg_similarity

In [14]:
column_consistency(correlation_tensor)

Column 0 skipped - zero vectors: True, constant vectors: False
  Matrix 0 - Unique values: 1, Sample: [0. 0. 0. 0. 0.]
  Matrix 1 - Unique values: 43, Sample: [0.         0.         0.51793104 0.25745493 0.09437038]
  Matrix 2 - Unique values: 1, Sample: [0. 0. 0. 0. 0.]
Column 1 skipped - zero vectors: True, constant vectors: False
  Matrix 0 - Unique values: 1, Sample: [0. 0. 0. 0. 0.]
  Matrix 1 - Unique values: 46, Sample: [0.02739906 0.         0.53769976 0.2828587  0.12145539]
  Matrix 2 - Unique values: 1, Sample: [0. 0. 0. 0. 0.]
Column 2 skipped - zero vectors: True, constant vectors: False
  Matrix 0 - Unique values: 46, Sample: [0.3755652  0.56189972 0.         0.         0.4716059 ]
  Matrix 1 - Unique values: 1, Sample: [0. 0. 0. 0. 0.]
  Matrix 2 - Unique values: 46, Sample: [0.3755652  0.56189972 0.         0.         0.4716059 ]
Column 3 skipped - zero vectors: True, constant vectors: False
  Matrix 0 - Unique values: 50, Sample: [0.58821529 0.72383368 0.27294779 0.    

([np.float64(0.8399811710702348), np.float64(0.6666671696426348)],
 np.float64(0.7533241703564348))